<a href="https://colab.research.google.com/github/AtharvBhat/PIAYN/blob/main/PIAYN_TextClassificationLRABaseline_New_4k.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Set-up environment

As usual, we first install HuggingFace Transformers, and Datasets.

In [ ]:
!pip install -q git+https://github.com/huggingface/transformers.git

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
     |████████████████████████████████| 895 kB 7.7 MB/s 
     |████████████████████████████████| 6.6 MB 46.8 MB/s 
     |████████████████████████████████| 77 kB 6.6 MB/s 
     |████████████████████████████████| 596 kB 62.0 MB/s 


In [ ]:
!pip install -q datasets

     |████████████████████████████████| 325 kB 9.0 MB/s 
     |████████████████████████████████| 1.1 MB 68.0 MB/s 
     |████████████████████████████████| 136 kB 76.3 MB/s 
     |████████████████████████████████| 212 kB 52.5 MB/s 
     |████████████████████████████████| 127 kB 69.9 MB/s 
     |████████████████████████████████| 94 kB 413 kB/s 
     |████████████████████████████████| 271 kB 54.5 MB/s 
     |████████████████████████████████| 144 kB 57.1 MB/s 
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [ ]:
!pip install ml_collections
!pip install absl-py

     |████████████████████████████████| 77 kB 4.8 MB/s 
  Created wheel for ml-collections: filename=ml_collections-0.1.1-py3-none-any.whl size=94524 sha256=42bb6d56af4dbd733fb86a34d548b89bf992640fdef665dfdac02edf40fac991
  Stored in directory: /root/.cache/pip/wheels/b7/da/64/33c926a1b10ff19791081b705879561b715a8341a856a3bbd2
Successfully built ml-collections


In [ ]:
!git clone https://github.com/google-research/long-range-arena.git
!git clone https://github.com/mlpen/Nystromformer.git
!git clone https://github.com/AtharvBhat/PIAYN.git

Cloning into 'long-range-arena'...
remote: Enumerating objects: 474, done.
remote: Counting objects: 100% (474/474), done.
remote: Compressing objects: 100% (191/191), done.
remote: Total 474 (delta 330), reused 423 (delta 282), pack-reused 0
Receiving objects: 100% (474/474), 153.28 KiB | 7.30 MiB/s, done.
Resolving deltas: 100% (330/330), done.
Cloning into 'Nystromformer'...
remote: Enumerating objects: 292, done.
remote: Counting objects: 100% (292/292), done.
remote: Compressing objects: 100% (225/225), done.
remote: Total 292 (delta 140), reused 147 (delta 42), pack-reused 0
Receiving objects: 100% (292/292), 10.82 MiB | 17.07 MiB/s, done.
Resolving deltas: 100% (140/140), done.
Cloning into 'PIAYN'...
remote: Enumerating objects: 22, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 22 (delta 6), reused 8 (delta 1), pack-reused 0
Unpacking objects: 100% (22/22), done.


## Set up output folders

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Set the task
task = 'text'

#Set Seq Length
max_length = 4000

In [ ]:
import os

baseDir = "/content/drive/MyDrive"
piaynDir = os.path.join(baseDir, "PIAYN")

if not os.path.exists(piaynDir):
  os.makedirs(piaynDir)

piaynTaskDir = os.path.join(piaynDir, task)

if not os.path.exists(piaynTaskDir):
  os.makedirs(piaynTaskDir)

piaynTaskModelDir = os.path.join(piaynTaskDir, "models")

if not os.path.exists(piaynTaskModelDir):
  os.makedirs(piaynTaskModelDir)

piaynTaskDataDir = os.path.join(piaynTaskDir, "data")

if not os.path.exists(piaynTaskDataDir):
  os.makedirs(piaynTaskDataDir)    

piaynTaskDataDirTokens = os.path.join(piaynTaskDataDir, str(max_length))

if not os.path.exists(piaynTaskDataDirTokens):
  os.makedirs(piaynTaskDataDirTokens)  

## Prepare data

Here we take a small portion of the IMDB dataset, a binary text classification dataset ("is a movie review positive or negative?").

In [ ]:
from datasets import load_dataset
#from Nystromformer.LRA.datasets import text
import pickle, numpy as np
import torch

torch.seed(42)

#copy over the custom text.py file to the Nystromformer location
!cp PIAYN/text.py Nystromformer/LRA/datasets

#Create Train test and dev MAP files for the imdb dataset
!python Nystromformer/LRA/datasets/text.py --max_length=$max_length

train_file = task+'.train.pickle'
dev_file = task+'.dev.pickle'
test_file = task+'.test.pickle'

!cp $train_file $piaynTaskDataDirTokens
!cp $dev_file $piaynTaskDataDirTokens
!cp $test_file $piaynTaskDataDirTokens

max_length:  4000
I0413 22:00:39.796064 139770126505856 dataset_builder.py:811] No config specified, defaulting to first: imdb_reviews/plain_text
2022-04-13 22:00:39.923578: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "NOT_FOUND: Could not locate the credentials file.". Retrieving token from GCE failed with "NOT_FOUND: Error executing an HTTP request: HTTP response code 404".
I0413 22:00:40.176463 139770126505856 dataset_info.py:434] Load pre-computed DatasetInfo (eg: splits, num examples,...) from GCS: imdb_reviews/plain_text/1.0.0
I0413 22:00:40.356762 139770126505856 dataset_info.py:361] Load dataset info from /tmp/tmphz8_kms1tfds
I0413 22:00:40.358932 139770126505856 dataset_info.py:405] Field info.config_name from disk and from code do not match. Keeping the one from code.
I0413 22:00:40.359103 139770126505856 dataset_info.py:405] Fie

In [ ]:
from Nystromformer.LRA.code import lra_config
from Nystromformer.LRA.code.dataset import LRADataset
#from Nystromformer.LRA.code.run_tasks import training_config
from torch.utils.data import DataLoader, RandomSampler

#get training config
training_config = lra_config.config[task]["training"]

#Update training config
#training_config["learning_rate"] = 0.00001
training_config["weight_decay"] = 0.1
training_config["eval_frequency"] = 1000

#Check Train Config
print('Training Config: ', training_config)

#get pre-defined model config
model_config = lra_config.config[task]['model']

#Check model Config
print('Model Config: ', model_config)

#Get the dataset
train_dataset = LRADataset(piaynTaskDataDirTokens + '/' + train_file, False)
val_dataset = LRADataset(piaynTaskDataDirTokens + '/' + dev_file, False)
test_dataset = LRADataset(piaynTaskDataDirTokens + '/' + test_file, False)

#Create DataLoader iterators
ds_iter = {
    "train":enumerate(DataLoader(train_dataset, 
                                 #Sample batches randomly for number of specified steps
                                 sampler = RandomSampler(train_dataset, 
                                                         replacement=True, 
                                                         num_samples= training_config["num_train_steps"]*lra_config.config[task]['dataset']['train']), 
                                 batch_size = training_config["batch_size"], 
                                 drop_last = True)),
    "dev":enumerate(DataLoader(val_dataset, batch_size = 32, drop_last = True)),
    "test":enumerate(DataLoader(test_dataset, batch_size = 32, drop_last = True)),
}


Training Config:  {'batch_size': 32, 'learning_rate': 0.0001, 'warmup': 8000, 'lr_decay': 'linear', 'weight_decay': 0.1, 'eval_frequency': 1000, 'num_train_steps': 20000, 'num_eval_steps': 781}
Model Config:  {'learn_pos_emb': True, 'tied_weights': False, 'embedding_dim': 64, 'transformer_dim': 64, 'transformer_hidden_dim': 128, 'head_dim': 32, 'num_head': 2, 'num_layers': 2, 'vocab_size': 512, 'max_seq_len': 4000, 'dropout_prob': 0.1, 'attention_dropout': 0.1, 'pooling_mode': 'MEAN', 'num_classes': 2}
Loaded /content/drive/MyDrive/PIAYN/text/data/4000/text.train.pickle... size=25000
Loaded /content/drive/MyDrive/PIAYN/text/data/4000/text.dev.pickle... size=25000
Loaded /content/drive/MyDrive/PIAYN/text/data/4000/text.test.pickle... size=25000


In [ ]:
#Check sizes of batches
batch = next((ds_iter['train']))
for k,v in batch[1].items():
  print(k,v.shape)

input_ids_0 torch.Size([32, 4024])
mask_0 torch.Size([32, 4024])
label torch.Size([32])


## Define model

Next, we define our model, and put it on the GPU.

In [ ]:
from transformers import PerceiverForSequenceClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
from transformers import PerceiverConfig
#get default perceiver config
configuration = PerceiverConfig()

#Update the Perceiver configurations with Preset model configs
#configuration.update(model_config)

#Print Updated Perceiver Configuration
print(configuration)

PerceiverConfig {
  "attention_probs_dropout_prob": 0.1,
  "audio_samples_per_frame": 1920,
  "cross_attention_shape_for_attention": "kv",
  "cross_attention_widening_factor": 1,
  "d_latents": 1280,
  "d_model": 768,
  "hidden_act": "gelu",
  "image_size": 56,
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 2048,
  "model_type": "perceiver",
  "num_blocks": 1,
  "num_cross_attention_heads": 8,
  "num_frames": 16,
  "num_latents": 256,
  "num_self_attends_per_block": 26,
  "num_self_attention_heads": 8,
  "output_shape": [
    1,
    16,
    224,
    224
  ],
  "qk_channels": null,
  "samples_per_patch": 16,
  "self_attention_widening_factor": 1,
  "train_size": [
    368,
    496
  ],
  "transformers_version": "4.19.0.dev0",
  "use_query_residual": true,
  "v_channels": null,
  "vocab_size": 262
}



In [ ]:
def initialize_model(config):
  #Initialize Model
  model = PerceiverForSequenceClassification(config)
  
  #Get Model Parameter Counts
  pytorch_total_params = sum(p.numel() for p in model.parameters())
  pytorch_total_params_Trainable = sum(p.numel() for p in model.parameters() if p.requires_grad)
  print('Total Parameters: ', pytorch_total_params, '\nTrainable Parameters: ', pytorch_total_params_Trainable)  

  return model#.to(device)                                          

In [ ]:
#Change the perceiver configurations to get total parameters within 10% of BERT
configuration.num_labels = 2
configuration.num_self_attends_per_block = 6
configuration.d_latents = 512
configuration.d_model = 512
configuration.num_layers = 6
configuration.max_position_embeddings = max_length + 24*np.power(2, int(max_length/1000)-1)



model = initialize_model(configuration)

Total Parameters:  15038978 
Trainable Parameters:  15038978


## Train the model

Here we train the model using native PyTorch.

In [ ]:
device

device(type='cuda')

In [16]:
from transformers import AdamW
from tqdm.notebook import tqdm
from sklearn.metrics import accuracy_score
from datasets import load_metric
import pandas as pd

best_score = 0 
prev_score = 0
maxPatience = 3
currentPatience = 0

#steps = int(training_config["num_train_steps"]/20000)
steps = training_config["num_train_steps"]

optimizer = AdamW(model.parameters(), 
                  lr = training_config["learning_rate"],
                  betas = (0.9, 0.999), 
                  eps = 1e-6, 
                  weight_decay = training_config["weight_decay"])

lr_scheduler = torch.optim.lr_scheduler.OneCycleLR(
    optimizer = optimizer,
    max_lr = training_config["learning_rate"],
    pct_start = training_config["warmup"] / training_config["num_train_steps"],
    anneal_strategy = training_config["lr_decay"],
    total_steps = training_config["num_train_steps"]
)

#amp_scaler = torch.cuda.amp.GradScaler() if model_config["mixed_precision"] else None

#initialize training summary
trainingSummary = pd.DataFrame(columns=['step', 'train_acc', 'val_acc'])


model.to(device)

#initialize training accuracy metric
train_accuracy = load_metric("accuracy")

for step in tqdm(range(steps)):  # Perform gradient updates for multiple steps
    
    model.train()
    
    #print("Step:", step)
    #for batch in tqdm(train_dataloader):
    batch = next(ds_iter['train'])[1]

    # get the inputs; 
    inputs = batch["input_ids_0"].to(device)
    attention_mask = batch["mask_0"].to(device)
    labels = batch["label"].to(device)

    # zero the parameter gradients
    optimizer.zero_grad()

    # forward + backward + optimize
    outputs = model(inputs=inputs, attention_mask=attention_mask, labels=labels)
    loss = outputs.loss
    loss.backward()
    optimizer.step()
    lr_scheduler.step()

    # evaluate
    predictions = outputs.logits.argmax(-1).cpu().detach().numpy()
    accuracy = accuracy_score(y_true=batch["label"].numpy(), y_pred=predictions)
    references = batch["label"].numpy()
    train_accuracy.add_batch(predictions=predictions, references=references)
    
    print(f"Loss: {loss.item()}, Accuracy: {accuracy}")

    #delete intermediate variables to free up GPU space
    del loss, outputs, inputs, attention_mask, labels, predictions, accuracy


    #Every 1000 steps validate and save model
    if (step+1)%training_config['eval_frequency']  == 0:
    #if (step+1)%2  == 0:
      
      model.eval()

      val_accuracy = load_metric("accuracy")

      #reset dev iterator
      ds_iter['dev'] = enumerate(DataLoader(val_dataset, batch_size = 32, drop_last = True))

      with torch.no_grad():
        for i, batch in tqdm(ds_iter['dev']):
              
          # get the inputs; 
          inputs = batch["input_ids_0"].to(device)
          attention_mask = batch["mask_0"].to(device)
          labels = batch["label"].to(device)

          #print('input shape: ', inputs.shape)
          #print('attention_mask shape: ', attention_mask.shape)
          #print('labels: ', labels.shape)

          # forward pass
          outputs = model(inputs=inputs, attention_mask=attention_mask)
          logits = outputs.logits 
          predictions = logits.argmax(-1).cpu().detach().numpy()
          references = batch["label"].numpy()
          val_accuracy.add_batch(predictions=predictions, references=references)

          #delete intermediate variables to free up GPU space
          del logits, outputs, inputs, attention_mask, labels, predictions, references
      
      #Compute val accuracy
      final_val_score = val_accuracy.compute()['accuracy']
      print("Validation Accuracy:", final_val_score)

      if final_val_score >= best_score:
        best_score = final_val_score
        torch.save(model.to('cpu').state_dict(), piaynTaskModelDir + '/trainedPerceiverClassifierToken'+str(max_length)+'.pkl')
        model.to(device)
      else:
        pass  

      if final_val_score <= prev_score:
        currentPatience += 1
        if currentPatience >= maxPatience:
          print('Patience Limit reached! Stopping early!')
          torch.save(model.to('cpu').state_dict(), piaynTaskModelDir + '/trainedPerceiverClassifierStep_' + str(step + 1) + 'Token' + str(max_length) + '.pkl')
          break  
      else:
        currentPatience = 0
      
      #Update prev_score
      prev_score = final_val_score

      #Compute training accuracy till now
      final_train_score = train_accuracy.compute()['accuracy']

      #Add to trainingSummary
      trainingSummary.loc[len(trainingSummary.index)] = [step+1, final_train_score, final_val_score]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


  0%|          | 0/20000 [00:00<?, ?it/s]

Loss: 0.6741583347320557, Accuracy: 0.625
Loss: 0.6764055490493774, Accuracy: 0.59375
Loss: 0.7053505182266235, Accuracy: 0.53125
Loss: 0.6449609398841858, Accuracy: 0.65625
Loss: 0.7514150738716125, Accuracy: 0.4375
Loss: 0.7403536438941956, Accuracy: 0.4375
Loss: 0.6942393779754639, Accuracy: 0.53125
Loss: 0.6574443578720093, Accuracy: 0.6875
Loss: 0.7293933629989624, Accuracy: 0.375
Loss: 0.6878337860107422, Accuracy: 0.5625
Loss: 0.6806654334068298, Accuracy: 0.6875
Loss: 0.6953498125076294, Accuracy: 0.40625
Loss: 0.692132294178009, Accuracy: 0.53125
Loss: 0.6904006004333496, Accuracy: 0.65625
Loss: 0.6965882778167725, Accuracy: 0.46875
Loss: 0.6941994428634644, Accuracy: 0.46875
Loss: 0.6913651823997498, Accuracy: 0.53125
Loss: 0.7015279531478882, Accuracy: 0.375
Loss: 0.6953904628753662, Accuracy: 0.5
Loss: 0.6906972527503967, Accuracy: 0.53125
Loss: 0.6902094483375549, Accuracy: 0.5625
Loss: 0.6954840421676636, Accuracy: 0.5
Loss: 0.7016793489456177, Accuracy: 0.40625
Loss: 0.6

0it [00:00, ?it/s]

Validation Accuracy: 0.6106754161331626
Loss: 0.684041440486908, Accuracy: 0.5625
Loss: 0.6637560725212097, Accuracy: 0.59375
Loss: 0.6839376091957092, Accuracy: 0.59375
Loss: 0.637818455696106, Accuracy: 0.6875
Loss: 0.7031393647193909, Accuracy: 0.53125
Loss: 0.6486344933509827, Accuracy: 0.625
Loss: 0.6506910920143127, Accuracy: 0.65625
Loss: 0.6493739485740662, Accuracy: 0.65625
Loss: 0.685965359210968, Accuracy: 0.625
Loss: 0.6919101476669312, Accuracy: 0.59375
Loss: 0.6639246940612793, Accuracy: 0.6875
Loss: 0.6216706037521362, Accuracy: 0.6875
Loss: 0.6582512855529785, Accuracy: 0.65625
Loss: 0.6357411742210388, Accuracy: 0.625
Loss: 0.6309353709220886, Accuracy: 0.6875
Loss: 0.670868992805481, Accuracy: 0.5625
Loss: 0.6625845432281494, Accuracy: 0.65625
Loss: 0.6095988154411316, Accuracy: 0.65625
Loss: 0.688427209854126, Accuracy: 0.5625
Loss: 0.7018542289733887, Accuracy: 0.5
Loss: 0.6744206547737122, Accuracy: 0.59375
Loss: 0.7090820074081421, Accuracy: 0.5
Loss: 0.6564021706

...

Validation Accuracy: 0.6228793213828425
Loss: 0.24007073044776917, Accuracy: 0.9375
Loss: 0.2756726145744324, Accuracy: 0.90625
Loss: 0.23097477853298187, Accuracy: 0.90625
Loss: 0.2531535029411316, Accuracy: 0.90625
Loss: 0.25540703535079956, Accuracy: 0.90625
Loss: 0.266975462436676, Accuracy: 0.875
Loss: 0.22024783492088318, Accuracy: 0.9375
Loss: 0.28454169631004333, Accuracy: 0.90625
Loss: 0.23406028747558594, Accuracy: 0.90625
Loss: 0.5699852705001831, Accuracy: 0.78125
Loss: 0.23653312027454376, Accuracy: 0.9375
Loss: 0.26960650086402893, Accuracy: 0.875
Loss: 0.36246567964553833, Accuracy: 0.8125
Loss: 0.2411271631717682, Accuracy: 0.9375
Loss: 0.33682751655578613, Accuracy: 0.8125
Loss: 0.29378220438957214, Accuracy: 0.875
Loss: 0.16090761125087738, Accuracy: 0.96875
Loss: 0.1455121487379074, Accuracy: 0.9375
Loss: 0.5983601808547974, Accuracy: 0.75
Loss: 0.41748717427253723, Accuracy: 0.78125
Loss: 0.16330412030220032, Accuracy: 0.9375
Loss: 0.27288350462913513, Accuracy: 0.9

0it [00:00, ?it/s]

Validation Accuracy: 0.6173175416133163
Loss: 0.18298833072185516, Accuracy: 0.9375
Loss: 0.40628325939178467, Accuracy: 0.84375
Loss: 0.28303417563438416, Accuracy: 0.9375
Loss: 0.30878522992134094, Accuracy: 0.90625
Loss: 0.2724873125553131, Accuracy: 0.90625
Loss: 0.3729129433631897, Accuracy: 0.8125
Loss: 0.31989583373069763, Accuracy: 0.84375
Loss: 0.290083110332489, Accuracy: 0.84375
Loss: 0.11479869484901428, Accuracy: 0.96875
Loss: 0.45021986961364746, Accuracy: 0.8125
Loss: 0.17503874003887177, Accuracy: 0.90625
Loss: 0.31230488419532776, Accuracy: 0.875
Loss: 0.3739605247974396, Accuracy: 0.84375
Loss: 0.35963374376296997, Accuracy: 0.8125
Loss: 0.32391130924224854, Accuracy: 0.90625
Loss: 0.19941313564777374, Accuracy: 0.96875
Loss: 0.21311314404010773, Accuracy: 0.9375
Loss: 0.20440007746219635, Accuracy: 0.9375
Loss: 0.4234921932220459, Accuracy: 0.84375
Loss: 0.2802256941795349, Accuracy: 0.875
Loss: 0.5375782251358032, Accuracy: 0.78125
Loss: 0.17283964157104492, Accurac

0it [00:00, ?it/s]

Validation Accuracy: 0.6163972471190781
Patience Limit reached! Stopping early!


In [17]:
#save training summary
trainingSummary.to_csv(piaynTaskModelDir + '/trainingSummaryToken'+str(max_length)+'.csv')

In [18]:
!nvidia-smi

Wed Apr 13 23:45:17 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   70C    P0    57W / 250W |  10249MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [19]:
del model
torch.cuda.empty_cache()
!nvidia-smi

Wed Apr 13 23:45:18 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    55W / 250W |   1279MiB / 16280MiB |    100%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

## Evaluate the model

Finally, we evaluate the model on the test set. We use the Datasets library to compute the accuracy.

In [20]:
from tqdm.notebook import tqdm
from datasets import load_metric

accuracy = load_metric("accuracy")

#load best performing model checkpoint
model = PerceiverForSequenceClassification(configuration)
model.load_state_dict(torch.load(piaynTaskModelDir + '/trainedPerceiverClassifierToken'+str(max_length)+'.pkl'))
model.to(device)

model.eval()

with torch.no_grad():
  for i, batch in tqdm(ds_iter['test']):
        
        # get the inputs; 
        inputs = batch["input_ids_0"].to(device)
        attention_mask = batch["mask_0"].to(device)
        labels = batch["label"].to(device)

        # forward pass
        outputs = model(inputs=inputs, attention_mask=attention_mask)
        logits = outputs.logits 
        predictions = logits.argmax(-1).cpu().detach().numpy()
        references = batch["label"].numpy()
        accuracy.add_batch(predictions=predictions, references=references)

        #delete intermediate variables to free up GPU space
        del logits, outputs, inputs, attention_mask, labels, predictions, references

final_score = accuracy.compute()
print("Accuracy on test set:", final_score['accuracy'])

0it [00:00, ?it/s]

Accuracy on test set: 0.646086747759283
